In [3]:
import os
import re

In [6]:
image_path = '/Users/arturgergert/Documents/Studium/Master/3.\ Semester/BigData_Analyseprojekt/satellite_imagery/data.nosync/three_band/6010_0_0.tif' 

id_full = os.path.basename(image_path)
id = re.split('_?[A-Z]?\.',id_full)[0]
print (f'id: {id} & imagepath: {image_path} & id_full: {id_full}')
#print(id_full)

id: 6010_0_0 & imagepath: /Users/arturgergert/Documents/Studium/Master/3.\ Semester/BigData_Analyseprojekt/satellite_imagery/data.nosync/three_band/6010_0_0.tif & id_full: 6010_0_0.tif


In [12]:
import tensorflow as tf
import numpy as np
import json
import requests
import os
import re
import matplotlib.pyplot as plt

from helpers.py import get_rgb_from_m_band, mask_to_polygons, mask_for_polygons
from helpers.py import stretch_n

N_Cls = 10
SIZE=128
MODEL_URI='http://localhost:8501/v1/models/unet:predict'
#CLASSES = ['Cat', 'Dog']

def get_prediction(image_path, image_size=160):

    #image_path is splitted to recieve the id
    id_full = os.path.basename(image_path)
    id = re.split('_?[A-Z]?\.',id_full)[0]
    print (f'id: {id} & imagepath: {image_path} & id_full: {id_full}')
    """ Ab hier der Ausschnitt aus predict_id"""

    img = get_rgb_from_m_band(image_id=id, inDir='data.nosync')
    x = stretch_n(img)

    cnv = np.zeros((960, 960, 8)).astype(np.float32)
    prd = np.zeros((N_Cls, 960, 960)).astype(np.float32)
    cnv[:img.shape[0], :img.shape[1], :] = x

    for i in range(0, 6):
        line = []
        for j in range(0, 6):
            line.append(cnv[i * image_size:(i + 1) * image_size, j * image_size:(j + 1) * image_size])

        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1

        """ Bis hier der Ausschnitt aus predict_id"""

        """ TF-Modelserver Ansprechen"""

        data = json.dumps({
            'instances': x.tolist()
        })
        response = requests.post(MODEL_URI, data=data.encode('utf-8'))
        result = json.loads(response.text)
        
    return result

print(get_prediction(image_path=image_path))

ModuleNotFoundError: No module named 'helpers.py'; 'helpers' is not a package